<a href="https://colab.research.google.com/github/JoSungHun/Deeplearning/blob/master/hw2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Assignment

* Due date: **2018/04/09 00:00** (will not accept late submission)
* Submittion format: notebook file which can be executed in Colab environment

* We want to build a multi-class classification model using Reuters dataset.

> ### Loading and preprocessing data

In [0]:
from keras.datasets import reuters

# Like IMDB, the argument num_words restricts the data to 
# the 10,000 most frequently occurring words 
(train_data, train_labels), (test_data, test_labels) = reuters.load_data(num_words=10000)

2113536/2110848 [==============================] - 1s 0us/step


In [0]:
import numpy as np

def vectorize_sequences(sequences, dimension=10000):
  results = np.zeros((len(sequences), dimension))
  for i, sequence in enumerate(sequences):
    results[i, sequence] = 1.
  return results

x_train = vectorize_sequences(train_data)
x_test = vectorize_sequences(test_data)

In [0]:
from keras.utils.np_utils import to_categorical

one_hot_train_labels = to_categorical(train_labels)
one_hot_test_labels = to_categorical(test_labels)

In [0]:
len(x_train)

8982

> ### Building the network

In [0]:
from keras import models
from keras import layers

def build_model():
  model = models.Sequential()
  model.add(layers.Dense(64, activation='relu', input_shape=(10000,)))
  model.add(layers.Dense(64, activation='relu'))
  model.add(layers.Dense(46, activation='softmax'))
  model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
  return model

In [0]:
model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

> ### Validation

* We employ *k-fold cross validation* for monitoring the performance of trained model.
* Write a code in the below to perform *10-fold cross validation*.
* **For each fold, save a model at every epoch in your Google Drive.**

In [0]:
from google.colab import drive#구글드라이브 마운트 및 파일경로 설정
from keras.callbacks import ModelCheckpoint
drive.mount('/content/gdrive')
filepath = '/content/gdrive/My Drive/hw2_14113359/hw2_model.{epoch:02d}.hdf5'
modelckpt = Modelcheckpoint(filepath=filepath) 

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
# write a code for 10-fold cross validation here
import numpy as np
k = 10
num_val_samples = len(train_data) // k
num_epochs = 50
all_train_acc = []
all_val_acc = []
for i in range(k):
  print('처리중인 폴드 #', i)
  
  val_data = train_data[i * num_val_samples: (i+1) * num_val_samples]
  val_targets = train_targets[i * num_val_samples: (i+1) * num_val_samples]
  
  partial_train_data = np.concatenate([train_data[:i * num_val_samples], train_data[(i + 1) * num_val_samples:]], axis=0)
  partial_train_targets = np.concatenate([train_targets[:i * num_val_samples], train_targets[(i + 1) * num_val_samples:]], axis=0)
    
  model = build_model()
  history = model.fit(partial_train_data,
                      partial_train_targets,
                      epochs=numepochs,
                      batch_size=512,
                      validation_data=(val_data, val_targets))
  
  train_acc = history.history['acc']
  val_acc = history.history['val_acc']
  
  all_train_acc.append(train_acc)
  all_val_acc.append(val_acc)

* Plotting the training and validation accuracy
  * To obtain the validation accuracy at the end of every epoch, just average the performances of all folds.

In [0]:
# write a code for plotting the training and validation accuracy

> ### Inference

* Find the best performance model by seeing the performance plot.
* Calculate the accuracy on test set using the best performance model.
  * Here, you should use a majority voting method to get the prediction for a test data point.
  * Specifically, given a test data point, get the predicted class from the trained model on each fold, and then decide the final predicted class by majority voting.
* **Do not retrain the model.**

In [0]:
# write a code here

In [11]:
! git clone https://github.com/JoSungHun/Deeplearning.git

Cloning into 'Deeplearning'...
remote: Enumerating objects: 20, done.
remote: Counting objects: 100% (20/20), done.
remote: Compressing objects: 100% (18/18), done.
remote: Total 20 (delta 4), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (20/20), done.
